### Setup Google Colab

In [ ]:
import os

if "COLAB_" in "".join(os.environ.keys()):
    try:
        import numpy

        get_numpy = f"numpy=={numpy.__version__}"
    except:
        get_numpy = "numpy"
    try:
        import subprocess

        is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except:
        is_tesla_t4 = False
    get_vllm, get_triton = (
        ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm", "triton")
    )
    !uv pip install --upgrade \
        "openpipe-art[backend]==0.4.8" langchain-core tenacity datasets "litellm[proxy]" "gql<4" "protobuf==5.29.5" {get_vllm} {get_numpy} --prerelease allow --no-cache-dir
    !uv pip install -qqq {get_triton}

Using Python 3.12.11 environment at: /usr
Resolved 241 packages in 7.71s
Prepared 123 packages in 32.54s
Uninstalled 54 packages in 1.65s
Installed 123 packages in 294ms
 - accelerate==1.10.0
 + accelerate==1.7.0
 + apscheduler==3.11.0
 + astor==0.8.1
 + asttokens==3.0.0
 + awscli==1.42.16
 + backoff==2.2.1
 + bitsandbytes==0.47.0
 + blake3==1.0.5
 + botocore==1.40.16
 - cachetools==5.5.2
 + cachetools==6.1.0
 + cbor2==5.7.0
 - cffi==1.17.1
 + cffi==2.0.0b1
 + colorama==0.4.6
 + comm==0.2.3
 + compressed-tensors==0.10.2
 + croniter==6.0.0
 - cupy-cuda12x==13.3.0
 + cupy-cuda12x==13.6.0
 + cut-cross-entropy==25.1.1
 - datasets==4.0.0
 + datasets==3.6.0
 - debugpy==1.8.15
 + debugpy==1.8.16
 - decorator==4.4.2
 + decorator==5.2.1
 + depyf==0.19.0
 - diffusers==0.34.0
 + diffusers==0.35.1
 + diskcache==5.6.3
 + dnspython==2.7.0
 - docutils==0.21.2
 + docutils==0.19
 + email-validator==2.2.0
 + eval-type-backport==0.2.2
 + executing==2.2.0
 - fastapi==0.116.1
 + fastapi==0.115.14
 + fastap

### Task Configuration


In [ ]:
# Required - Used for generating training inputs and RULER evaluation
OPENAI_API_KEY = ""
BASE_URL = "https://pd67dqn1bd.execute-api.eu-west-1.amazonaws.com"

# Optional - Enables metric logging
WANDB_API_KEY = ""

# Choose the base model to train
BASE_MODEL = "Qwen/Qwen3-0.6B"

# Model configuration
MODEL_NAME = "promo-matcher-90-v0"  # Name for your trained model
PROJECT_NAME = "prosus-assignment-colab"  # Project name for tracking

# Training configuration
TRAINING_CONFIG = {
    "num_training_inputs": 90,  # Number of training inputs to generate, 10 users
    "num_testing_inputs": 10, # Number of testing inputs
    "groups_per_step": 2,  # Inputs to process per training step
    "num_epochs": 3,  # Number of times through all data
    "rollouts_per_group": 5,  # Different responses per input (for RULER comparison), we have 10 promos so 5 is probably better than 3
    "learning_rate": 1e-5,  # Learning rate
    "max_training_steps": None,  # Maximum training steps (set to None for no limit)
}

NUM_TEST_INPUTS = 5  # Number of test inputs to generate
RULER_MODEL = "openai/gpt-4.1"  # Model for RULER evaluation
# SYSTEM_PROMPT_GENERATION_MODEL = "openrouter/moonshotai/kimi-k2"
# INPUT_GENERATION_MODEL = "openrouter/moonshotai/kimi-k2"

# GPU configuration (for T4 — keep these as-is unless you have a reason to change them)
MAX_SEQ_LENGTH = 4096  # Maximum sequence length
GPU_MEMORY_UTILIZATION = 0.7  # GPU memory usage (0.0-1.0)

### Task Description

In [ ]:

MATCH_PROMOTION_TASK_DESCRIPTION = """
You are selecting exactly ONE restaurant promotion that maximizes expected user engagement (click/claim/order) for the provided user profile.
Use ONLY these user features for each user in your matching decision: eat_habit, eat_time_pattern, diet, app_behavior, meal_frequency, snack_frequency,
cooking_skill, activity_level, breakfast_time, lunch_time, dinner_time, app_usage_hours, notification_preference,
goal_oriented, social_features_user.

There are 10 candidate promotions to choose from:
1. Flash Feast: "Get 50% off all orders between 2 PM and 5 PM on weekdays. This promotion is designed to increase orders during typically slow periods."
2. Mystery Meal Monday: "Let us pick your dinner! Get a surprise main course from a top-rated local restaurant for a fixed price of $10."
3. Snap & Share Sunday: "Post a photo of your meal ordered through our app on social media, tag us, and use our special hashtag for a chance to win a $50 gift card. This encourages user-generated content and social media engagement."
4. Two-for-One Tuesdays: "Buy one main course and get a second one of equal or lesser value for free from participating restaurants."
5. Weekday Lunch Deal: "To attract the weekday lunch crowd, we're offering a special combo: a main dish, a side, and a drink for a discounted price."
6. Refer-a-Friend Rewards: "Invite a friend to the app and you both get $10 off your next order."
7. Themed Dinner Experience: "This week, explore the tastes of Italy with our 'Taste of Tuscany' promotion, featuring exclusive dishes and discounts from local Italian restaurants."
8. Loyalty Program Launch: "Earn points for every dollar you spend. Accumulate 500 points and receive a $15 credit. This encourages repeat business."
9. Late-Night Bites: "Craving a midnight snack? Get free delivery on all orders placed after 10 PM."
10. Family Meal Bundle: "Perfect for a family night in, get two large pizzas, a side of garlic bread, and a 2-liter soda for a special bundled price."

Decision rules:
1. Primary objective: Choose the promo that best fits the user’s habits/preferences and is most likely to be acted on.
2. Time alignment: Look at the user's preferences for time alignment, specific promotions match much better to users with a preferred eat time.
3. Behavior fit: Certain users have behavioral features that fit better with specific promotions.
4. Diet compatibility: Do not choose promotions that likely conflict with diet
5. Avoid: Picking promotions whose core premise is clearly misaligned, for example a late-night offer for an early-to-bed user.

Output format (policy must follow exactly):
{"promotion_id": "P#", "promotion_name": "...", "reason": "<=30 words"}
"""

SYSTEM_PROMPT = MATCH_PROMOTION_TASK_DESCRIPTION

### Training Data Preperation

In [ ]:
import pandas as pd, json

df = pd.read_csv("./fake_people.csv")

PROFILE_COLS = ["eat_habit","eat_time_pattern","diet","app_behavior","meal_frequency",
                "snack_frequency","cooking_skill","activity_level","breakfast_time",
                "lunch_time","dinner_time","app_usage_hours","notification_preference",
                "goal_oriented","social_features_user"]

records = []
training_inputs = []

for _, row in df.iterrows():
    user_id = str(row["id"])
    profile  = {k: row[k] for k in PROFILE_COLS}
    inp = json.dumps({"user_profile": profile}, ensure_ascii=False, separators=(",",":"))
    training_inputs.append(inp)
    records.append({"user_id": user_id, "input": inp})

N_TRAIN = TRAINING_CONFIG["num_training_inputs"]
N_TEST = TRAINING_CONFIG["num_testing_inputs"]
train_records = records[:N_TRAIN] # Number of training samples
test_records  = records[-N_TEST:] # Hold out the last 10 records for testing

print(train_records[0])

{'user_id': 'user_001', 'input': '{"user_profile":{"eat_habit":"Emotional eater","eat_time_pattern":"Brunch lover (10 AM-12 PM)","diet":"DASH","app_behavior":"Weekend warrior","meal_frequency":"3 times/day","snack_frequency":"Rarely","cooking_skill":"Expert","activity_level":"Moderately active","breakfast_time":"07:30","lunch_time":"14:00","dinner_time":"18:15","app_usage_hours":"Throughout day","notification_preference":"Off","goal_oriented":true,"social_features_user":false}}'}


### Training the model

In [ ]:
import os
import random
from typing import List

import torch
import weave
from dotenv import load_dotenv
from litellm import acompletion
from pydantic import BaseModel, Field

import art
from art.local import LocalBackend
from art.rewards import ruler_score_group
from art.utils import iterate_dataset
from art.utils.litellm import convert_litellm_choice_to_openai

load_dotenv()

# Mute LiteLLM logging bug
import logging, os, warnings
logging.getLogger("LiteLLM").setLevel(logging.CRITICAL)
os.environ["LITELLM_LOGGING"] = "False"
os.environ["LITELLM_LOG"] = "False"
warnings.filterwarnings("ignore", message=".*'deprecated' attribute.*Field\\(\\).*")

# Required
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
    raise ValueError(
        "OPENROUTER_API_KEY is required for data generation and RULER evaluation."
    )

# Optional
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
else:
    print("WANDB_API_KEY is not set. We'll skip logging metrics to Weights & Biases.")

# =========== Model Creation Code ===========

random.seed(42)

# Declare the model
model = art.TrainableModel(
    name=MODEL_NAME,
    project=PROJECT_NAME,
    base_model=BASE_MODEL,
)

# To run on a T4, we need to override some config defaults.
if torch.cuda.get_device_properties(0).major < 8:
    model._internal_config = art.dev.InternalModelConfig(
        init_args=art.dev.InitArgs(
            max_seq_length=MAX_SEQ_LENGTH,
        ),
        engine_args=art.dev.EngineArgs(
            enforce_eager=True,
            gpu_memory_utilization=GPU_MEMORY_UTILIZATION,
        ),
    )

# Initialize the server
if torch.cuda.get_device_properties(0).major < 8:
    backend = LocalBackend(
        in_process=True,
        path="./.art",
    )
else:
    backend = LocalBackend()

# Register the model with the local Backend
await model.register(backend)

print("Model created!")
print("Base model:", BASE_MODEL)
print("Model name:", MODEL_NAME)
print("Project name:", PROJECT_NAME)

# ============ Rollout Function Code =============

if os.getenv("WANDB_API_KEY", ""):
    weave.init(PROJECT_NAME, settings={"print_call_link": False})


class TaskInput(BaseModel):
    step: int
    input_text: str
    user_id: str  # to keep track of user


@weave.op
async def rollout(model: art.Model, task_input: TaskInput) -> art.Trajectory:
    """Execute a single rollout for the custom task"""

    traj = art.Trajectory(
        reward=0.0,
        messages_and_choices=[],
        metadata={
            "step": task_input.step,
            "input": task_input.input_text,
            "user_id": task_input.user_id,
        },
    )

    # Build the conversation
    traj.messages_and_choices = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": task_input.input_text},
    ]

    # Get model response
    if model.trainable:
        litellm_model_name = f"hosted_vllm/{model.name}"
    else:
        litellm_model_name = model.name

    response = await acompletion(
        model=litellm_model_name,
        base_url=model.inference_base_url,
        api_key=model.inference_api_key,
        temperature=0.7,
        max_tokens=4096,
        messages=traj.messages(),
        caching=False,
    )

    # Add the model's response to the trajectory
    traj.messages_and_choices.append(
        convert_litellm_choice_to_openai(response.choices[0])
    )

    return traj


print("\nRollout function defined!")


# Test RULER with example outputs for a text formalization task
test_input = "hey can u send me the report asap? thx"

base_messages = [
    {"role": "system", "content": "Convert informal text to formal business language."},
    {"role": "user", "content": test_input},
]

good_trajectory = art.Trajectory(
    messages_and_choices=[
        *base_messages,
        {
            "role": "assistant",
            "content": "Could you please send me the report at your earliest convenience? Thank you.",
        },
    ],
    reward=0,
)

mediocre_trajectory = art.Trajectory(
    messages_and_choices=[
        *base_messages,
        {"role": "assistant", "content": "Can you send me the report soon? Thanks."},
    ],
    reward=0,
)

bad_trajectory = art.Trajectory(
    messages_and_choices=[
        *base_messages,
        {"role": "assistant", "content": "hey send report quick thx"},
    ],
    reward=0,
)

sample_group = art.TrajectoryGroup(
    trajectories=[good_trajectory, mediocre_trajectory, bad_trajectory]
)

# RULER will score these based on how well they accomplish the task
# Allow ten retries in case of API rate limiting
for i in range(10):
    try:
        judged_group = await ruler_score_group(
            sample_group,
            RULER_MODEL,
            debug=True,
            extra_litellm_params={"api_base": BASE_URL}
        )
        break
    except Exception as e:
        print(f"Error scoring group: {e}")
        continue

assert judged_group is not None

# Display rankings
sorted_trajectories = sorted(
    judged_group.trajectories, key=lambda t: t.reward, reverse=True
)
for rank, traj in enumerate(sorted_trajectories, 1):
    messages = traj.messages()
    print(f"\nRank {rank}: Score {traj.reward:.3f}")
    print(f"  Response: {messages[-1]['content']}")


# ============ Training Loop =============

# Convert training inputs to TaskInput objects
training_task_inputs = [
    TaskInput(step=0, input_text=r["input"], user_id=r["user_id"])
    for r in train_records]

# Create training iterator
training_iterator = iterate_dataset(
    training_task_inputs,
    groups_per_step=TRAINING_CONFIG["groups_per_step"],
    num_epochs=TRAINING_CONFIG["num_epochs"],
    initial_step=await model.get_step(),
)

print(f"Starting training with {len(training_task_inputs)} inputs...")
print(f"Training for {TRAINING_CONFIG['num_epochs']} epoch(s)")
print(
    f"Generating {TRAINING_CONFIG['rollouts_per_group']} responses per input for RULER to compare"
)
print(
    "\nWhy multiple responses? RULER needs to compare different attempts to learn what's good!"
)

for batch in training_iterator:
    print(
        f"\nTraining step {batch.step}, epoch {batch.epoch}, epoch step {batch.epoch_step}"
    )
    print(f"Batch contains {len(batch.items)} inputs")

    # Create trajectory groups for this batch
    groups = []
    for task_input in batch.items:
        # Update step number
        task_input.step = batch.step

        # Generate multiple responses for each input (RULER will compare these)
        groups.append(
            art.TrajectoryGroup(
                (
                    rollout(model, task_input)
                    for _ in range(TRAINING_CONFIG["rollouts_per_group"])
                )
            )
        )

    # Gather all trajectory groups
    finished_groups = await art.gather_trajectory_groups(
        groups,
        pbar_desc="Generating responses",
        max_exceptions=TRAINING_CONFIG["rollouts_per_group"] * len(batch.items),
    )

    # Use RULER to score each group
    judged_groups = []
    for group in finished_groups:
        # Allow ten retries in case of API rate limiting
        judged_group = None
        for i in range(10):
            try:
                judged_group = await ruler_score_group(
                    group,
                    RULER_MODEL,
                    debug=False,
                     extra_litellm_params={"api_base": BASE_URL}
                )
                break
            except Exception as e:
                print(f"Error scoring group: {e}")
                continue
        assert judged_group is not None
        judged_groups.append(judged_group)

    # Train on the scored trajectories
    await model.delete_checkpoints()
    await model.train(
        judged_groups,
        config=art.TrainConfig(learning_rate=TRAINING_CONFIG["learning_rate"]),
        _config={"logprob_calculation_chunk_size": 8},
    )

    print(f"Completed training step {batch.step}")

    # Stop after configured steps (if limit is set)
    if (
        TRAINING_CONFIG["max_training_steps"]
        and batch.step >= TRAINING_CONFIG["max_training_steps"]
    ):
        print(
            f"Reached maximum training steps ({TRAINING_CONFIG['max_training_steps']})"
        )
        break

print("\n✅ Training completed!")

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

  warnings.warn(

  warnings.warn(

  | |_| | '_ \/ _` / _` |  _/ -_)

wandb: Currently logged in as: alexzh (alexzh-tu-delft) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin

INFO 08-24 01:42:26 [__init__.py:235] Automatically detected platform cuda.
Model created!
Base model: Qwen/Qwen3-0.6B
Model name: promo-matcher-90-v0
Project name: prosus-assignment-colab

Rollout function defined!


[RULER] Pretty-printed LLM choice JSON:

{
    'scores': [
        {
            'trajectory_id': '1',
            'explanation': 'This output fully converts the informal text into formal business language. It uses 
polite phrasing and appropriate formality, fulfilling the goal completely and efficiently.',
            'score': 1.0
        },
        {
            'trajectory_id': '2',
            'explanation': 'This output makes the text more polite and removes some informality, but it is less 
formal than ideal for business language. It gets partial credit for making progress, but does not fully achieve the
goal.',
            'score': 0.6
        },
        {
            'trajectory_id': '3',
            'explanation': 'This output leaves the original message almost unchanged, retaining informal phrasing 
and not converting to formal business language. Minimal progress toward the goal.',
            'score': 0.1
        }
    ]
}


Rank 1: Score 1.000
  Response: Could you please send me the report at your earliest convenience? Thank you.

Rank 2: Score 0.600
  Response: Can you send me the report soon? Thanks.

Rank 3: Score 0.100
  Response: hey send report quick thx
Starting training with 90 inputs...
Training for 3 epoch(s)
Generating 5 responses per input for RULER to compare

Why multiple responses? RULER needs to compare different attempts to learn what's good!


...

Completed training step 133

Training step 134, epoch 2, epoch step 44
Batch contains 2 inputs


Generating responses:   0%|          | 0/10 [00:00<?, ?it/s]

No "val/reward" metric found in history
Deleted checkpoint ./.art/prosus-assignment-colab/models/promo-matcher-90-v0/checkpoints/0133
Packed 10 trajectories into 6 sequences of length 6144


train:   0%|          | 0/6 [00:00<?, ?it/s]

Completed training step 134

✅ Training completed!


### Testing the model

In [ ]:
print(f"\n🧪 Testing the trained model on {len(test_records)} held-out inputs:\n")
print("=" * 80)

for i, r in enumerate(test_records):
    print(f"\nTest {i}:")
    print(f"Input: {r['input']}")

    test_task_input = TaskInput(step=999, input_text=r["input"], user_id=r["user_id"])
    result_trajectory = await rollout(model, test_task_input)

    # Extract the model's response
    messages = result_trajectory.messages()
    model_response = messages[-1]["content"] if messages else "No response"

    print(f"Model output: {model_response}")
    print("-" * 80)

print("\n🎉 Testing completed!")
print(f"\nYour model '{MODEL_NAME}' has been trained to: {SYSTEM_PROMPT}")
print("\nTo use this model in production:")
print("1. The model checkpoint is saved in ./.art/")
print("2. You can load it using the vLLM library")
print(
    "3. Or continue training with more examples by adjusting the configuration at the top"
)


🧪 Testing the trained model on 10 held-out inputs:


Test 0:
Input: {"user_profile":{"eat_habit":"Fast eater","eat_time_pattern":"Late night snacker","diet":"No restrictions","app_behavior":"Meal planner","meal_frequency":"3 times/day","snack_frequency":"Never","cooking_skill":"Doesn't cook","activity_level":"Extremely active","breakfast_time":"08:45","lunch_time":"14:45","dinner_time":"17:00","app_usage_hours":"Throughout day","notification_preference":"Off","goal_oriented":false,"social_features_user":true}}
Model output: <think>
Okay, let's see. The user's eat_habit is a fast eater, so they might be interested in something quick. Their eat_time_pattern is a late night snacker, so they probably want something late. Their diet is no restrictions, so no issues there. App_behavior is a meal planner, so they might need a promotion that's meal-related. Meal_frequency is 3 times/day, so they might want a meal planner promotion. Snack_frequency is never, so maybe a snack promotion. Cooking

### Download the LoRa checkpoint

In [ ]:
# Path to your checkpoint folder (adjust if needed)
CKPT_DIR = "/content/.art/prosus-assignment-colab/models/promo-matcher-90-v0/checkpoints/0135"

# Create a compressed archive
!tar -czf /content/promo-matcher-90-checkpoint-0135.tar.gz -C "$(dirname "$CKPT_DIR")" "$(basename "$CKPT_DIR")"

# Download to your machine
from google.colab import files
files.download("/content/promo-matcher-90-checkpoint-0135.tar.gz")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Model Evaluation

### Model Comparison w/ RULER as judge